In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime as dt
import numpy as np

def load_haps_positions(
    csv_path: str,
    when: dt.datetime,
    tol: timedelta = timedelta(hours=1.5)
) -> list[tuple[float, float, float]]:
    # filter HAPS positions within time tolerance window
    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    mask = (df["timestamp"] >= when - tol) & (df["timestamp"] <= when + tol)
    pos_df = df.loc[mask, ["lat", "lon", "alt"]]
    return list(pos_df.itertuples(index=False, name=None))

def plot_haps_on_world_map(
    csv_path: str,
    when: dt.datetime,
    tol: timedelta = timedelta(hours=1.5),
    land_shp_path: str = "map/ne_10m_land/ne_10m_land.shp",
    lakes_shp_path: str = "map/ne_10m_lakes/ne_10m_lakes.shp",
    rivers_shp_path: str = "map/ne_10m_rivers_lake_centerlines/ne_10m_rivers_lake_centerlines.shp",
    maritime_shp_path: str = "map/ne_10m_ocean/ne_10m_ocean.shp",
    coastline_shp_path: str = "map/ne_10m_coastline/ne_10m_coastline.shp",
    figsize: tuple = (12, 8)
):
    # load shapefiles for basemap
    land = gpd.read_file(land_shp_path)
    lakes = gpd.read_file(lakes_shp_path)
    rivers = gpd.read_file(rivers_shp_path)
    ocean = gpd.read_file(maritime_shp_path)
    coastline = gpd.read_file(coastline_shp_path)

    # get HAPS positions
    haps_positions = load_haps_positions(csv_path, when, tol)
    if not haps_positions:
        raise ValueError("No HAPS positions found in the given time window")

    # split into separate lists
    lats, lons, alts = zip(*haps_positions)

    # create plot
    fig, ax = plt.subplots(figsize=figsize)
    # plot ocean first (background)
    ocean.plot(ax=ax, color=np.array([122, 213, 255]) / 255.0, edgecolor='none')
    # plot land areas
    land.plot(ax=ax, color=np.array([230, 255, 230]) / 255.0, edgecolor='#888888', linewidth=0.3)
    # overlay lakes and rivers
    lakes.plot(ax=ax, color=np.array([122, 213, 255]) / 255.0, edgecolor='none')
    # draw coastlines for clarity
    coastline.plot(ax=ax, linewidth=0.5)

    # plot HAPS positions
    ax.scatter(
        lons, lats,
        s=20,               # marker size
        c='red',            # marker color
        marker='x',         # marker shape
    )

    # set labels and title
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("HAPS Positions on World Map")
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])

    # set aspect so that degrees are equal
    ax.set_aspect('equal', adjustable='box')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_haps_on_world_map(
    csv_path="map/haps_positions.csv",
    when=dt.datetime(2020, 12, 31, 0)
)

In [6]:
import os
from pathlib import Path
import datetime as dt
from datetime import timedelta
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import imageio              # v2 API for get_writer & imread
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# ---------------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------------
CSV_PATH   = "map/haps_positions.csv"

LAND_SHP   = "map/ne_10m_land/ne_10m_land.shp"
LAKE_SHP   = "map/ne_10m_lakes/ne_10m_lakes.shp"
RIVER_SHP  = "map/ne_10m_rivers_lake_centerlines/ne_10m_rivers_lake_centerlines.shp"
OCEAN_SHP  = "map/ne_10m_ocean/ne_10m_ocean.shp"
COAST_SHP  = "map/ne_10m_coastline/ne_10m_coastline.shp"

FRAME_DIR  = Path("mp4_frames")         # temporary PNG frames folder
MP4_PATH   = Path("haps_2020_6h.mp4")   # output MP4 file path

TOL        = timedelta(hours=1.5)       # ±1.5h time window
START      = dt.datetime(2020, 1, 1, 0)
END        = dt.datetime(2020, 12, 31, 18)
STEP       = timedelta(hours=6)
FPS        = 10                          # frames per second

# ---------------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------------
def load_haps_positions(csv_path: str,
                        when: dt.datetime,
                        tol: timedelta = TOL) -> list[tuple[float, float, float]]:
    """Return list of (lat, lon, alt) within |when − timestamp| ≤ tol."""
    df = pd.read_csv(csv_path, parse_dates=["timestamp"])
    mask = (df["timestamp"] >= when - tol) & (df["timestamp"] <= when + tol)
    pos_df = df.loc[mask, ["lat", "lon", "alt"]]
    return list(pos_df.itertuples(index=False, name=None))

def init_basemap():
    """Load all shapefiles once and return as dict."""
    return {
        "land": gpd.read_file(LAND_SHP),
        "lakes": gpd.read_file(LAKE_SHP),
        "rivers": gpd.read_file(RIVER_SHP),
        "ocean": gpd.read_file(OCEAN_SHP),
        "coast": gpd.read_file(COAST_SHP),
    }

def draw_world_map(ax, shapes):
    """Draw base layers: ocean, land, lakes, coastlines."""
    shapes["ocean"].plot(ax=ax, color=np.array([122,213,255])/255.0, edgecolor="none")
    shapes["land"].plot(ax=ax, color=np.array([230,255,230])/255.0,
                        edgecolor="#888888", linewidth=0.3)
    shapes["lakes"].plot(ax=ax, color=np.array([122,213,255])/255.0, edgecolor="none")
    shapes["coast"].plot(ax=ax, linewidth=0.5)

def render_frame(when: dt.datetime, shapes, frame_path: Path):
    """Render one PNG frame at time 'when', annotate date bottom-right."""
    positions = load_haps_positions(CSV_PATH, when, TOL)
    fig, ax = plt.subplots(figsize=(12, 8))
    draw_world_map(ax, shapes)

    if positions:
        lats, lons, _ = zip(*positions)
        ax.scatter(lons, lats,
                   s=20, c="red", marker="x",
                   label=f"HAPS ({len(positions)})")
        ax.legend(loc="lower left")

    # add date text in format Year-Day-Month at bottom-right
    date_text = when.strftime("%Y-%d-%m")
    ax.text(0.99, 0.01, date_text,
            transform=ax.transAxes,
            ha="right", va="bottom",
            fontsize=12,
            bbox=dict(facecolor="white", alpha=0.6, edgecolor="none"))

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_aspect("equal", adjustable="box")
    plt.tight_layout()
    fig.savefig(frame_path, dpi=150)
    plt.close(fig)

# ---------------------------------------------------------------------------
# Parallel worker setup
# ---------------------------------------------------------------------------
def worker_init():
    """Initialize global SHAPES in worker process."""
    global SHAPES
    SHAPES = init_basemap()

def worker_render(args):
    """Worker function to render a frame given (when, idx). Skip if exists."""
    when, idx = args
    frame_file = FRAME_DIR / f"frame_{idx:04d}.png"
    if not frame_file.exists():
        render_frame(when, SHAPES, frame_file)
    return str(frame_file)

# ---------------------------------------------------------------------------
# Main pipeline (parallelized)
# ---------------------------------------------------------------------------
def build_mp4_parallel():
    FRAME_DIR.mkdir(exist_ok=True)
    total_frames = ((END - START) // STEP) + 1

    # prepare list of (datetime, index) for each frame
    args_list = [(START + STEP * i, i) for i in range(total_frames)]

    # use all CPU cores to render frames in parallel
    with Pool(processes=cpu_count(), initializer=worker_init) as pool:
        frames_str = list(tqdm(
            pool.imap(worker_render, args_list),
            total=total_frames, desc="Rendering frames"
        ))

    frames = [Path(fp) for fp in frames_str]  # ordered list of frame paths

    # assemble MP4 from rendered frames using imageio v2
    print("Encoding MP4…")
    with imageio.get_writer(str(MP4_PATH), fps=FPS, format="FFMPEG", quality=8) as writer:
        for fp in tqdm(frames, desc="Writing MP4"):
            img = imageio.imread(str(fp))
            writer.append_data(img)

    print(f"MP4 saved to: {MP4_PATH.absolute()}")

if __name__ == "__main__":
    build_mp4_parallel()


Rendering frames: 100%|██████████| 1464/1464 [00:00<00:00, 50353.11it/s]


Encoding MP4…


Writing MP4:   0%|          | 0/1464 [00:00<?, ?it/s]/tmp/ipykernel_613527/1880902288.py:132: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(str(fp))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1800, 1200) to (1808, 1200) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
Writing MP4: 100%|██████████| 1464/1464 [00:43<00:00, 33.50it/s]


MP4 saved to: /home/hslyu/research/SSIR/scripts/haps_2020_6h.mp4
